## 图像预处理

In [1]:
import numpy as np
from PIL import Image
from pylab import *
import tensorflow as tf

In [3]:
S = 256
crop_size = 224

image_path = r'C:\Users\iceic\Desktop\1.png'
or_image = Image.open(image_path)
or_image.show()

image_w,image_h = or_image.size  #获得图片的长宽

scale = S/(min(image_h,image_w)) # 0.4 将图片最小边缩小为 224

#计算等比缩放后图像的长宽
new_w = np.int(image_w*scale)
new_h = np.int(image_h*scale)

print(new_w,new_h) #256 341

resize_image = or_image.resize((new_w,new_h), Image.BICUBIC) #使用线性差值方法缩放

max_offset_width = np.random.randint(low = 0,high = new_w - crop_size + 1,dtype = 'int32')

max_offset_height = np.random.randint(low = 0,high = new_h - crop_size + 1,dtype = 'int32')


384 256


In [4]:
crop_image = resize_image.crop((max_offset_width, max_offset_height, max_offset_width + crop_size, max_offset_height + crop_size))
#裁剪图片
crop_image.show()

In [5]:
print(crop_image.size)

(224, 224)


In [6]:
transpose_image = crop_image.transpose(Image.FLIP_LEFT_RIGHT)
#水平翻转

transpose_image.show()

In [7]:
r,g,b = transpose_image.split()
#分离三个通道

In [8]:
pic = Image.merge('RGB',(g,r,b))
#交换 g r 通道
pic.show()

## 使用 tensorflow 进行数据增强

In [10]:
def _smallest_size_at_least(height, width, smallest_side):
    """
    计算将图片最小边缩放为目标尺寸时，图像的宽高为多少，即返回目标的宽高
    函数实现 将图像最小边缩放为 smallest_side 时,宽度和高度分别为多少
    param height 原始图像高度
    param width 原始图像宽度
    param smallest_size
    return e 缩放之后图像的高度和宽度
    """
    #把宽高都转为float类型
    smallest_side = tf.convert_to_tensor(smallest_side, dtype = tf.int32)
    height = tf.to_float(height)
    width = tf.to_float(width)
    smallest_side = tf.to_float(smallest_side)
    
    #计算缩放比例
    scale = tf.cond(tf.greater(height, width), lambda:smallest_side/width,
                                               lambda:smallest_side/height)
    #tf.cond(pred, fn1, fn2, name=None)  Return :either fn1() or fn2() based on the boolean predicate `pred`
    
    new_height = tf.cast(tf.rint(height * scale), tf.int32)
    new_width = tf.cast(tf.rint(width * scale), tf.int32)
    #tf.cast()函数的作用是执行 tensorflow 中张量数据类型转换，比如读入的图片如果是int8类型的，一般在要在训练前把图像的数据格式转换为float32
    #tf.rint函数用于返回最接近X的元素整数
    
    return new_height,new_width

In [12]:
def aspect_preserving_resize(image, smallest_side):
    """
    函数实现将图片等比缩放，最小边长为smallest_side
    param image: 3维张量
    param smallest_side: 等比缩放后最小边长
    return: 等比变化的张量
    """
    smallest_side = tf.convert_to_tensor(smallest_side,dtype=tf.int32) #转为tf张量
    shape = tf.shape(image) #获得张量尺寸
    
    #获得高与宽
    height = shape[0]
    width = shape[1]
    
    #获得需要缩放到smallest_side尺寸的 新尺寸
    new_height,new_width = _smallest_size_at_least(height, width, smallest_side)
    
    #tf.image.convert_image_dtype 图像归一化  归为tf.float32
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    
    
    resized_image = tf.image.resize_images(image,[new_heigh, new_width],
                                          method = tf.image.ResizeMethod.BICUBIC)
    return resized_image

In [16]:
def random_crop(image, crop_height, crop_width):
    original_shape = tf.shape(image)
    
    #判断original_shape的维度是否为3，若是则不会抛出异常
    rank_assertion = tf.Assert(tf.equal(tf.rank(image),3),
                              ['Rank of image must be equal to 3.'])
   
    """
    Assert(
    condition,
    data,
    summarize=None,
    name=None
    )
    
    If condition evaluates to false, print the list of tensors in data. summarize determines how many entries of the tensors to print.
    """
    
    
    
    #cropped_shape = [224,224,3]
    with tf.control_dependencies([rank_assertion]):
        cropped_shape = tf.stack([crop_height,crop_width,original_shape[2]])
    
    
    #判断原图大小与剪切图像块的大小，若原图小于图像块尺寸则抛出异常
    #tf.greater >   ;   tf.greater_equal   >=
    size_assertion = tf.Assert(tf.logical_and(
                                                tf.greater_equal(orginal_shape[0], crop_height),
                                                tf.greater_equal(orginal_shape[1], crop_width),
                                                ['Crop size greater than the image size.'])
    
    
                               
                               
                               
    #确定最大裁剪范围
    max_offset_height = tf.reshape(original_shape[0] - crop_height + 1,[])
    max_offset_width = tf.reshape(original_shape[1] - crop_width + 1,[])
    
    
    
    

SyntaxError: invalid syntax (<ipython-input-16-180389341c4f>, line 38)